In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import gridspec

In [ ]:
df_porte_operadoras = pd.read_csv('../entrada/porte_operadoras.csv',encoding="ISO-8859-1")
df_cadu = pd.read_excel('../entrada/CADU.xls',encoding="ISO-8859-1")
df_cadop = pd.read_csv("../entrada/CooperativasMedicasAtivas_CADOP.txt",sep="\t",encoding="ISO-8859-1")

### FILTRANDO OPERADARAS ATIVAS DO ARQUIVO DE CADASTROS

In [ ]:
df_cadu_operadoras  = df_cadu[(df_cadu["STATUS"] == "Ativo") & (df_cadu["TIPO DE ATUAÇÃO"] == 'OPERADORA')]

## INFORMAÇÕES RELEVANTES DO CONJUNTO DE DADOS

In [ ]:
print("Quantidade de operadoras ativas:\t",df_cadu_operadoras['REGISTRO ANS'].count())
print("Total Beneficiários:\t\t\t",df_cadu_operadoras['QUANTIDADE DE BENEFICIÁRIOS'].sum())
print("Total Cooperados:\t\t\t",df_cadu_operadoras['COOPERADOS TOTAL'].sum())
print("Total de funcionários:\t\t\t",df_cadu_operadoras['QUANTIDADE DE FUNCIONÁRIOS'].sum())

In [ ]:
def gera_grafico_group_by(serie_valores,titulo) :
    
    fig = plt.figure(figsize=(18,20))
    gs = gridspec.GridSpec(9,1)
    
    ax1 = fig.add_subplot((gs[0:-3]))
    serie_valores.plot(kind='pie', y = "Classes", ax=ax1, autopct='%1.1f%%', 
    startangle=90,   legend = True, fontsize=16,)
    ax1.title.set_text(titulo)
    ax1.title.set_size(30)
    

    ax2 = fig.add_subplot((gs[7:]))
    serie_valores.plot(kind='bar',fontsize=16)
    ax2.title.set_text(titulo)
    ax2.title.set_size(30)

    #Ajuste das labels dos gráfico de barras
    for xticks in ax2.get_xticklabels():
        xticks.set_rotation(0)    

    rects = ax2.patches
    for rect, label in zip(rects, serie_valores.values):
        height = rect.get_height()
        ax2.text(rect.get_x() + rect.get_width() / 2, height, label,
                ha='center', va='bottom')
    plt.legend(loc='best',fontsize="small")
#     plt.tight_layout()
    plt.subplots_adjust(hspace = 0.5)
    nome_arquivo = titulo.replace(".","").replace(" ","_")
    plt.savefig("../graficos/"+nome_arquivo)
    plt.show()
    
    
   


## Quantidade de Operadoras por Porte

In [ ]:
serie_quantidade_operadoras_por_porte = df_cadu_operadoras.groupby(by=['PORTE'])['REGISTRO ANS'].count()

In [ ]:
gera_grafico_group_by(serie_quantidade_operadoras_por_porte,"Quantidade Operadora por porte")

## Operadora por região

In [ ]:
serie_quantidade_operadoras_por_regiao = df_cadu_operadoras.groupby(by=['REGIÃO'])['REGISTRO ANS'].count()
gera_grafico_group_by(serie_quantidade_operadoras_por_regiao,"Quantidade operadoras por região")

In [ ]:
df_cadu.groupby(['PORTE','REGIÃO'])['REGISTRO ANS'].count()

## Região por porte da operadora

In [ ]:
for porte in df_cadu_operadoras["PORTE"].unique():
    serie_porte_por_regiao = df_cadu_operadoras[df_cadu_operadoras["PORTE"] == porte].groupby(by=["REGIÃO"])['REGISTRO ANS'].count()
    gera_grafico_group_by(serie_porte_por_regiao, "Distribuição porte operadora por região - "+porte)

## Beneficiários das operadoras por região 

In [ ]:
serie_quantidade_beneficiarios_por_regiao =  df_cadu_operadoras.groupby(by=['REGIÃO'])['QUANTIDADE DE BENEFICIÁRIOS'].sum()

In [ ]:
gera_grafico_group_by(serie_quantidade_beneficiarios_por_regiao, "Quantidade beneficiários por região")

## Porte Operadoras por Região

In [ ]:
for regiao in df_cadu_operadoras["REGIÃO"].unique():
    serie_operadoras_por_regiao = df_cadu_operadoras[df_cadu_operadoras["REGIÃO"] == regiao].groupby(by=["PORTE"])['REGISTRO ANS'].count()
    gera_grafico_group_by(serie_operadoras_por_regiao, "Porte operadoras da região "+regiao)

## Todas as operadores por região

In [ ]:
total_beneficiarios_por_ano = df_porte_operadoras.groupby(by=["ano"],as_index=False)['beneficiarios'].sum()
total_beneficiarios_por_ano.set_index("ano")
total_beneficiarios_por_ano.columns = ["ano", "total"]


In [ ]:
media_beneficiarios_por_ano = df_porte_operadoras.groupby(by=["ano"],as_index=False)['beneficiarios'].mean()
media_beneficiarios_por_ano.set_index("ano")
media_beneficiarios_por_ano.columns = ["ano", "media"]
media_beneficiarios_por_ano['media'] = media_beneficiarios_por_ano['media'].map(lambda x: '%2.0f' % x).astype(float)

In [ ]:
df_beneficiarios_por_ano = total_beneficiarios_por_ano.merge(media_beneficiarios_por_ano,on="ano")

In [ ]:
plt.figure(figsize=(20,10))
ax1 = df_beneficiarios_por_ano['media'].plot(secondary_y=True, color='red',linewidth=5.0)
ax2 = df_beneficiarios_por_ano['total'].plot(kind='bar')
ax = plt.gca()
ax.set_xticklabels(df_beneficiarios_por_ano['ano'])

plt.legend(loc='best')
plt.tight_layout()
plt.show()

In [ ]:
def sinalizar_mudanca_porte(x):
    ano_anterior = x['ano'] - 1
    df_ano_anterior = df_porte_operadoras[(df_porte_operadoras['ano'] == ano_anterior) & (df_porte_operadoras['Registro_ANS'] == x['Registro_ANS'])].squeeze()
    variacao = "0"
    if (len(df_ano_anterior) > 0):
        porte_anterior = df_ano_anterior['porte']
        porte_atual = x['porte']
        if ((porte_anterior == "Pequeno Porte" and porte_atual in ["Médio Porte",'Grande Porte']) or  (porte_anterior == "Médio Porte" and porte_atual=='Grande Porte')):
            return  "subiu"
        elif ((porte_anterior == "Grande Porte" and porte_atual in ["Médio Porte","Pequeno Porte"]) or  (porte_anterior == "Médio Porte" and porte_atual=="Pequeno Porte")):
            return "abaixou"
        else:
            return "manteve"

In [215]:
df_porte_operadoras['variacao'] = df_porte_operadoras.apply(lambda x: sinalizar_mudanca_porte(x),axis=1)

In [217]:
df_cadu.columns

Index(['REGISTRO ANS', 'RAZÃO SOCIAL', 'TIPO DE COOPERATIVA',
       'TIPO DE ATUAÇÃO', 'GRAU DA COOPERATIVA', 'HIERARQUIA 1',
       'HIERARQUIA 2', 'HIERARQUIA 3', 'CEP', 'ESTADO', 'REGIÃO', 'LOCALIDADE',
       'CNPJ', ' DATA DA FUNDAÇÃO', ' ÁREA DE AÇÃO',
       'QUANTIDADE DE FUNCIONÁRIOS', 'COOPERADOS TOTAL',
       'QUANTIDADE DE BENEFICIÁRIOS', 'PORTE', 'STATUS', 'DATA DE INATIVAÇÃO'],
      dtype='object')

In [220]:
df_cadop.columns

Index(['Registro_ANS', 'Razao_Social', 'Nome_Fantasia', 'Regiao_de_Atuacao'], dtype='object')

In [221]:
df_porte_operadoras.columns

Index(['Registro_ANS', 'ano', 'beneficiarios', 'porte', 'variacao'], dtype='object')

In [222]:
df_cadu_todas_operadoras  = df_cadu[(df_cadu["TIPO DE ATUAÇÃO"] == 'OPERADORA')]

In [230]:
df_cadu_todas_operadoras = df_cadu_todas_operadoras[['REGISTRO ANS',' DATA DA FUNDAÇÃO','STATUS', 'DATA DE INATIVAÇÃO']]

In [232]:
df_cadu_todas_operadoras[].head()

,REGISTRO ANS,DATA DA FUNDAÇÃO,STATUS,DATA DE INATIVAÇÃO
0,355721,18/12/1967,Ativo,NaN
1,335690,17/12/1970,Ativo,NaN
2,315729,14/12/1970,Ativo,NaN
3,331872,13/02/1971,Ativo,NaN
4,343269,11/03/1971,Ativo,NaN
